In [1]:
from exllamav2 import(
    ExLlamaV2,
    ExLlamaV2Config,
    ExLlamaV2Cache,
    ExLlamaV2Tokenizer
)

from exllamav2.generator import (
    ExLlamaV2BaseGenerator,
    ExLlamaV2Sampler
)

model_dir = "/home/robbie/sdiff/exllamav2/vntl-7b-v0.3.1-hf-8.0bpw-exl2"

config = ExLlamaV2Config()
config.model_dir = model_dir
config.prepare()

model = ExLlamaV2(config)

cache = ExLlamaV2Cache(model, lazy = True)
model.load_autosplit(cache)

tokenizer = ExLlamaV2Tokenizer(config)

# Initialize generator

generator = ExLlamaV2BaseGenerator(model, cache, tokenizer)
generator.warmup()

In [2]:
import sqlite3
db = sqlite3.connect("../blume.db")

In [5]:
import re

names = {
    'Mary': 'メアリ',
    'Narrator': None,
    '???': '？？？',
    'Daniela': 'ダニエラ',
    'Victor': 'ヴィクトル',
    'Auguste': 'オーギュスト',
    'Ilia': 'イリア',
    'Richard': 'リチャード',
    'Jacob': 'ヤコブ',
    'Auguste\'s Voice': 'オーギュストの声',
    'Virginia': 'バージニア',
    'Klaus': 'クラウス',
    'Stefan': 'ステファン',
    'Relm': 'レルム',
    'Gerald': 'ジェラルド'
}

prompt_start = """<s><<START>>
Name: Mary (メアリ) | Gender: Female
Name: Daniela Brancusi (ダニエラ・ブランクーシ) | Gender: Female
Name: Victor Friedrich (ヴィクトル・フリードリヒ) | Gender: Male
Name: Auguste Muller (オーギュスト・ミュラー) | Gender: Male
Name: Ilia Cantemir (イリア・カンテミール) | Gender: Female
Name: Richard Cantemir (リチャード・カンテミール) | Gender: Male | Aliases: Onii-chan (お兄ちゃん)
Name: Jacob Cantemir (ヤコブ・カンテミール) | Gender: Male | Aliases: mayor (村長)
Name: Virginia Moreno (バージニア・モレノ) | Gender: Female
Name: Klaus (クラウス) | Gender: Male
Name: Stefan (ステファン) | Gender: Male
Name: Relm (レルム) | Gender: Male
Name: Gerald Villbervint (ジェラルド・ヴィルベルヴィント) | Gender: Male
"""

exp = re.compile("(?:\[(?:.+?)\]: )?(.+)", re.S)

seen = []

settings = ExLlamaV2Sampler.Settings()
settings.temperature = 0.6
#settings.top_k = 1
settings.top_p = 0.9

scriptid = 102

n = 0
for address, speaker, line in db.cursor().execute("SELECT address, speaker, line FROM lines WHERE scriptid = ?", (scriptid,)):
    line = line.replace("#Name[1]", "メアリ")
    jpspeaker = names[speaker]
    
    if jpspeaker is not None:
        jpspeaker = "[{}]: ".format(jpspeaker)
    else:
        jpspeaker = ""
    
    if speaker != "Narrator":
        speaker = "[{}]: ".format(speaker)
    else:
        speaker = ""
    
    nucleus = "<<JAPANESE>>\n{}{}\n<<ENGLISH>> (fidelity = high)\n{}".format(jpspeaker, line, speaker)
    while True:
        prompt = ''.join([prompt_start, *seen, nucleus])
        inputs = tokenizer.encode(prompt, encode_special_tokens=True)[0]
        if len(inputs) <= 3840:
            break
        seen.pop(0)

    tl = generator.generate_simple(prompt, settings, 256, seed=0, encode_special_tokens=True)[len(prompt)-3:].strip()
    nucleus += tl + "\n"
    seen.append(nucleus)
    print(speaker + tl)
    db.cursor().execute("INSERT OR REPLACE INTO translations VALUES('vntl-20240206', ?, ?, ?)", (scriptid, address, tl))
    db.commit()

[Mary]: 「...No!」
[Mary]: 「Haaah, haaah, haaah...」
I was sitting up in bed.
[Mary]: 「What's going on?」
I looked around to check.
The desk had a chair, the wall had an image, and I was wearing my church clothes...
This was my familiar room.
[Mary]: 「Haaah, haaah, haaah... D-Did I sleepwalk?」
I definitely had been dreaming. But it was such a realistic one.
I didn't want to wake up from that dream, but I couldn't wake up either.
I felt glad that it was only a dream, but I remembered the screams, the heat of the flames, and the warmth of human skin.
[Mary]: 「Ungh...」
I shivered.
[Mary]: 「It's cold...」
I realized for the first time that my nightgown was soaked.
[Mary]: 「I need to change. I'll catch a cold...」
I got out of bed and started changing. I happened to look outside the window and saw pitch blackness.
I had forgotten to close the shutters.
The endless darkness made it impossible to see anything. What could be waiting for me on the other side?
Or maybe there was nothing at all.
[Mary]